In [ ]:
#going to scrape the 2019 WARN notices and compare to 2020

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [ ]:
# assign the NY WARN notice url to a variable

In [30]:
url = 'https://labor.ny.gov/app/warn/default.asp?warnYr=2019'

In [31]:
#headers
headers = {'accept-encoding': 'deflate', 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.87 Safari/537.36'}

In [32]:
# get request
response = requests.get(url, headers = headers)

In [33]:
print(response)

<Response [200]>


In [34]:
type(response.text)

str

In [35]:
soup = BeautifulSoup(response.text, 'html.parser')
#soup

In [36]:
table = soup.find("table")
#table

In [37]:
rows = soup.find_all("tr")
#rows

In [38]:
#WARN notices for 2019
len(rows) 

346

In [42]:
# make an array called 'results'
results = []
event_numbers = set()

# loop through the rows using a for loop. each row here is a company
for row in rows:
    # grab the anchor tag (the link tag) in the row and then grab the href attribute from the tag
    a = row.find("a")['href']
    
    # concatenate the root url from above with this href attribute and assign to a variable called 'company_url'
    company_url = f'https://labor.ny.gov/app/warn/{a}'
    #company_url = 'https://labor.ny.gov/app/warn/details.asp?id=7341'
    #print(company_url)
    
    # make a get request to the company url assign the response to a variable called 'company_response'
    company_response = requests.get(company_url, headers = headers)
    #print(company_response)
    
    # parse the response text and assign output to a variable called 'company_soup'
    company_soup = BeautifulSoup(company_response.text, 'html.parser')

    # grab the first table on the page
    company_table = company_soup.find("table")

    # unwrap all of the spans
    
    # loop through all of the p tags
    paragraphs = company_table.find_all("p")
    skip = False
    for p in paragraphs:
        # grab all of the values we want
        text = p.get_text('\n').replace('\xa0', '')
        if 'Date of Notice:' in text:
            split_notice_date = text.split(":")
            #print(split_notice_date)
            if len(split_notice_date) == 3:
                print(split_notice_date)
                notice_date = split_notice_date[2].strip().split()[0].strip()
            else:
                notice_date = text.split(":")[1].strip().split()[0].strip().replace(',', '').replace(';', '')
            print(notice_date)
        elif 'Event Number:' in text:
            event_number = text.split(":")[1].strip()
            if event_number in event_numbers:
                print('repeated event number')
                print(company_url)
                skip = True
                break
            else:
                event_numbers.add(event_number)
        elif 'Reason Stated for Filing:' in text:
            reason = text.split(":")[1].strip()
            #print(reason)
        elif 'Company:' in text:
            split_company = [x.strip() for x in text.split('\n')]
            #print(split_company)
            company = split_company[1].strip()
            address = ' '.join(split_company[2:])
#             print(company)
#             print(address)
        elif 'County:' in text:
            county = f'{text.split(":")[1].strip().split("|")[0].strip()} County'
            #print(county)
        elif 'Phone:' in text:
            phone = text.split(":")[1].strip()
            #print(phone)
        elif 'Business Type:' in text:
            business_type = text.split(":")[1].strip().replace('Restaurants', 'Restaurant')
            #print(business_type)
        elif 'Number Affected:' in text:
            affected = text.split(":")[1].strip().split(" ")[0].strip().split('\n')[0].strip().replace(',', '').replace('(', '')
            if affected in ['------', '-----', '----']:
                affected = 0
        elif 'Total Employees:' in text:
            total_employees = text.split(":")[1].strip().split(" ")[0].strip().replace(',', '')
            if total_employees in ['------', '-----', '----']:
                total_employees = 0
            #print(total_employees)
        elif 'Layoff Date:' in text:
            #print(text)
            layoff_date = text.split(":")[1].strip().split(" ")[0].strip().split(" ")[0].strip()
            #print(layoff_date)
        elif ('Reason for Dislocation:' in text):
            dislocation = text.split(":")[1].strip()
            #print(dislocation)
        elif ('Union:' in text):
            union = text.split(":")[1].strip()
            #print(union)
        elif ('Classification:' in text):
            classification = text.split(":")[1].strip()
            #print(classification)
            
    # store values in a result object
    if not skip:
        result = {
            'notice_date': notice_date,
            'event_number': event_number,
            'reason': reason,
            'company': company,
            'address': address,
            'county': county,
            'phone': phone,
            'business_type': business_type,
            'affected': affected,
            'total_employees': total_employees,
            'layoff_date': layoff_date,
            'dislocation': dislocation,
            'union': union,
            'classification': classification
         }

        # append result object to results
        results.append(result)
    #break

12/24/2019
['Date of Notice', ' 10/16/2019\nAmendment', ' 12/23/2019']
12/23/2019
['Date of Notice', ' 9/18/2019\nAmendment', ' 12/20/2019']
12/20/2019
['Date of Notice', ' 10/7/2019  \nAmendment', ' 12/20/2019']
12/20/2019
12/20/2019
12/12/2019
['Date of Notice', '11/22/2019(Amended', '12/12/2019)']
12/12/2019)
12/17/2019
12/10/2019
12/12/2019
12/12/2019
12/10/2019
11/19/2019
9/30/2019
12/9/2019
['Date of Notice', ' 9/9/2019 \nAmended', ' 12/9/2019']
12/9/2019
12/11/2019
12/6/2019
12/5/2019
12/4/2019
11/27/2019
['Date of Notice', ' 11/22/2019\nAmended', ' 12/3/2019']
12/3/2019
repeated event number
https://labor.ny.gov/app/warn/details.asp?id=7024
12/6/2019
12/5/2019
12/3/2019
12/3/2019
12/2/2019
11/27/2019
10/22/2019
['Date of Notice', ' 9/23/2019 \nAmended', ' 11/26/2019']
11/26/2019
12/3/2019
11/27/2019
11/25/2019
11/22/2019
11/22/2019
repeated event number
https://labor.ny.gov/app/warn/details.asp?id=7010
11/22/2019
11/25/2019
['Date of Notice', ' 7/13/2018 \nAmended', ' 11/25/201

In [43]:
# wrap results in a dataframe
df = pd.DataFrame(results)
df.shape

(300, 14)

In [44]:
df['affected'].unique()

array(['45', '165', '80', '130', '15', '48', '423', '3', '1', '61', '49',
       '16', '83', '34', '68', '156', '201', '40', '11', '4', '82', '27',
       '99', '13', '63', '54', '21', '118', '450', '160', '87', '12',
       '128', '44', '78', '84', '66', '23', '2', '136', '91', '52', '33',
       '126', '86', '96', '29', '42', '39', '64', '72', '303', '7', '60',
       '352', '47', '117', '32', '189', '35', '6', '26', '270', '328',
       '41', '57', '104', '67', '107', '569', '51', '50', '62', '65',
       '115', '24', '106', '79', '73', '30', '38', '55', '58', '265',
       '28', '145', '97', '105', '0', '10', '59', '98', '18', '71', '14',
       '325', '143', '5', '85', '134', '244', '88', '170', '172', '129',
       '56', '81', '228', '110', '286', '285', '857', '263', '92', '77',
       '135', '95', '9', '76', '388', '121', '43', '144', '273', '193',
       '216', '75', '89', '31', '20', '36', '528', '367', '112', '441',
       '100', '17', '22', '46', '207', '155', '70'], dtype=

In [45]:
# output dataframe to a csv
df.to_csv('../data/warn_2019.csv', index=False)

In [46]:
df

,notice_date,event_number,reason,company,address,county,phone,business_type,affected,total_employees,layoff_date,dislocation,union,classification
0,12/24/2019,2019-0198,Plant Layoff,"Timeless Décor, LLC (LCO Destiny DBA Timeless ...","22419 Fisher Road Watertown, NY 13601",Jefferson County,(315) 782-5759,A supplier of Picture Frames,45,81,The,Economic,The employees are not represented by a union.,Plant layoff
1,12/23/2019,2019-0102,Plant Closing,Barneys New York,"575 5th Avenue New York, NY 10017",New York County,(212) 450-8606,Retail,165,165,Employment,Economic,New York New Jersey Regional Joint Board Worke...,Plant Closing
2,12/20/2019,2019-0066,Plant Closing,"General Parts Distribution, LLC d/b/a Carquest...","215 Business Park Drive Armonk, NY 10504",Westchester County,(860) 375-1514,Auto parts distribution,80,80,Of,Economic,Teamsters Local 202,Plant Closing
3,12/20/2019,2019-0092,Plant Closing,"Somos Healthcare Providers, Inc.","519 Eighth Avenue, 14th Floor New York, NY 10118",New York County,(516) 642-4068,Community care and advocate community provider,130,130,Separations,New York State contract expired,The employees are not represented by a union.,Plant Closing
4,12/20/2019,2019-0197,Plant Unit Closing,"Regeneron Healthcare Solutions, Inc., a wholly...","1 Rockwood Rd. Sleepy Hollow, NY 10591",Westchester County,(914) 847-7128,Biotechnology company,15,0,2/18/2020,Termination of all field and supporting staff ...,The employees are not represented by a union.,Plant Unit Closing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1/6/2019,2018-0215,Plant Closing,Morrison Healthcare Food and Nutrition Service...,"462 Grider Street Buffalo, NY 14215",Erie County,(914) 426-8240,National food and nutrition services company,172,172,4/6/2019,Contractual Loss,"CSEA Local 1000, AFSCME, AFL-CIO",Plant Closing
296,12/27/2018,2018-0213,Plant Closing,Kmart Store (Unit 07677),"121 Bolivar Road Wellsville, NY 14895",Allegany County,(847) 286-1427,Retail Store,88,88,Employment,Economic,The employees are not represented by a union.,Plant Closing
297,12/27/2018,2018-0214,Plant Closing,Kmart Store (Unit 04928),"49 Dix Avenue Ext. Queensbury, NY 12804",Warren County,(847) 286-1427,Retail Store,60,60,Employment,Economic,The employees are not represented by a union.,Plant Closing
298,12/27/2018,2018-0211,Plant Closing,"Sears, Roebuck and Co. Full Line Store (Unit 0...","10 Miracle Mile Dr. Rochester, NY 14623",Monroe County,(847) 286-1427,Retail Store,70,70,Employment,Economic,The employees are not represented by a union.,Plant Closing
